# Scrapbook for Home-depot Question

## Imports

In [1]:
# Data Wrangling
import numpy as np
import pandas as pd

In [128]:
# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

In [3]:
# Misc
import os
import re
from pprint import pprint as pp

In [4]:
# Machine Learning
import nltk
import spacy
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Config

In [64]:
pd.options.mode.chained_assignment = None

In [5]:
data_path = "../../scrap/KAG_home-depot/"

In [6]:
stemmer = SnowballStemmer('english')

In [15]:
nlp = spacy.load('en')

In [16]:
nlp_tag = spacy.load('en', disable=['parser', 'ner'])

## Functions

In [17]:
def str_stemmer(s):
    return " ".join([stemmer.stem(word) for word in s.lower().split()])

In [18]:
def str_common_word(str1, str2):
    return sum(int(str2.find(word)>=0) for word in str1.split())

In [102]:
def process_data(df):
    df['product_title'] = df['product_title'].apply(lambda x: str_stemmer(x))
    df['search_term'] = df['search_term'].apply(lambda x: str_stemmer(x))
    df['product_description'] = df['product_description'].apply(lambda x: str_stemmer(x))
    df['query_len'] = df['search_term'].apply(lambda x: len(x.split()))
    df['product_info'] = df['search_term']+"\t"+df['product_title']+"\t"+df['product_description']
    df['word_in_title'] = df['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
    df['word_in_description'] = df['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
    return df

In [20]:
def get_pos_counts(search_terms_list):
    st = nlp(search_terms_list[0])
    pos_st1 = pd.Series([x.pos_ for x in st]).value_counts()
    del st
    st = nlp(search_terms_list[1])
    pos_st2 = pd.Series([x.pos_ for x in st]).value_counts()
    del st
    pos_tot = (pos_st1 + pos_st2).sort_values()

In [21]:
# # Plot
# ax = pos_tot.plot(kind='bar')
# ax.set_title('Count of POS Tags', )
# ax.set_xlabel('POS Tag')
# ax.set_ylabel('Count')
# plt.savefig('pos_tags_bar.png', dpi=500, bbox_inches="tight")

In [22]:
def lemm(word, nlp):
    return nlp(word)[0].lemma_

## Import data

In [103]:
os.listdir('../../scrap/')
csvs = [f for f in os.listdir(data_path) if re.search(".csv$", f)]
data_dict = {}
for csv in csvs:
    data_dict[csv.split()[0]] = pd.read_csv(data_path+csv, encoding='latin1')
test_df = data_dict['test.csv']
train_df = data_dict['train.csv']
prod_desc = data_dict['product_descriptions.csv']
attributes = data_dict['attributes.csv']
sample_sub = data_dict['sample_submission.csv']

['attributes.csv', 'sample_submission.csv', 'product_descriptions.csv', 'train.csv', 'df_test.csv', 'test.csv', 'df_train.csv']


## Data Handling

In [122]:
attr = attributes.dropna(how='all')
attr[['name','value']] = attr[['name','value']].fillna('')
attr['product_uid'] = attr['product_uid'].apply(lambda x: int(x))
attr['name'] = attr['name'].apply(lambda x: '' if "Bullet" in x else x)
attr['Attributes'] = attr['name'] + '\t' + attr['value'] + '\n'
attr = attr.drop(['name','value'], axis=1).groupby('product_uid').sum().reset_index()
df_train = pd.merge(train_df, prod_desc, how='left', on='product_uid').drop('id', axis=1).merge(attr, on='product_uid')

In [139]:
df_train.head()

,product_uid,product_title,search_term,relevance,product_description,Attributes
0,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ...",\tVersatile connector for various 90Â° connect...
1,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ...",\tVersatile connector for various 90Â° connect...
2,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...,"Application Method\tBrush,Roller,Spray\nAssemb..."
3,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...,Bath Faucet Type\tCombo Tub and Shower\nBuilt-...
4,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...,Bath Faucet Type\tCombo Tub and Shower\nBuilt-...


## Investigation

In [138]:
df_train['relevance'].hist(bins=8)

In [42]:
# Get docs
pt_doc = nlp(df_train.iloc[0,1])
st_doc = nlp(df_train.iloc[0,2])
pd_doc = nlp(df_train.iloc[0,4])

In [43]:
df_train['search_term'].apply(lambda x: len(x.split())).mean()

3.1592072042880095

In [44]:
uid = 100002
print(df_train[(df_train['relevance']==3.0) & (df_train['product_uid']==uid)].head()['product_title'].values[0])
print()
print(df_train[(df_train['relevance']==3.0) & (df_train['product_uid']==uid)].head()['search_term'].values[0])
print()
print(df_train[(df_train['relevance']==3.0) & (df_train['product_uid']==uid)].head()['product_description'].values[0])
print()
pp(attr.loc[uid, 'name'])
pp(attr.loc[uid, 'value'].values)

BEHR Premium Textured DeckOver 1-gal. #SC-141 Tugboat Wood and Concrete Coating

deck over

BEHR Premium Textured DECKOVER is an innovative solid color coating. It will bring your old, weathered wood or concrete back to life. The advanced 100% acrylic resin formula creates a durable coating for your tired and worn out deck, rejuvenating to a whole new look.  For the best results, be sure to properly prepare the surface using other applicable BEHR products displayed above.California residents: see&nbsp;Proposition 65 informationRevives wood and composite decks, railings, porches and boat docks, also great for concrete pool decks, patios and sidewalks100% acrylic solid color coatingResists cracking and peeling and conceals splinters and cracks up to 1/4 in.Provides a durable, mildew resistant finishCovers up to 75 sq. ft. in 2 coats per gallonCreates a textured, slip-resistant finishFor best results, prepare with the appropriate BEHR product for your wood or concrete surfaceActual paint 

In [45]:
print([x for x in pd_doc if not x.is_stop])

[Not, angles, joints, stronger, ,, provide, consistent, ,, straight, corners, ., Simpson, Strong, -, Tie, offers, wide, variety, angles, sizes, thicknesses, handle, light, -, duty, jobs, projects, structural, connection, needed, ., Some, bent, (, skewed, ), match, project, ., For, outdoor, projects, moisture, present, ,, use, ZMAX, zinc, -, coated, connectors, ,, provide, extra, resistance, corrosion, (, look, ", Z, ", end, model, number).Versatile, connector, 90, connections, home, repair, projectsStronger, angled, nailing, screw, fastening, aloneHelp, ensure, joints, consistently, straight, strongDimensions, :, 3, ., x, 3, ., x, 1, -, 1/2, ., Made, 12-Gauge, steelGalvanized, extra, corrosion, resistanceInstall, 10d, common, nails, #, 9, x, 1, -, 1/2, ., Strong, -, Drive, SD, screws]


In [46]:
search_terms = ' '.join(df_train['search_term'].values)

In [47]:
l = len(search_terms)
search_terms_list = [
    search_terms[0:int(l/2)],
    search_terms[int(l/2):]
]

In [126]:
words_st = ' '.join(df_train['search_term'].values).split()

In [156]:
len(words_st)/5000 * 9.64 / 60

7.518975066666667

In [158]:
len(words_st)/5000 * 32 / 60

24.959253333333333

In [165]:
import time
start = time.time()
# list((map(lambda x: lemm(x, nlp_tag), words_st[:5000])))
stems = list(map(lambda x: (stemmer.stem(x),), words_st))
print(time.time()-start)

0.0787668228149414


Okay. That was decent. However, let's work on this and do better!

Attributes wasn't used...I think there must be something here. Also only stems were taken...can we not extract keywords?

In [191]:
nltk.pos_tag(nltk.word_tokenize(' '.join(attr.loc[100001, 'value'].values)))

[('12', 'CD'),
 ('Simpson', 'NNP'),
 ('Strong-Tie', 'NNP'),
 ('Galvanized', 'NNP'),
 ('Steel', 'NNP'),
 ('Versatile', 'NNP'),
 ('connector', 'NN'),
 ('for', 'IN'),
 ('various', 'JJ'),
 ('90Â°', 'CD'),
 ('connections', 'NNS'),
 ('and', 'CC'),
 ('home', 'NN'),
 ('repair', 'NN'),
 ('projects', 'NNS'),
 ('Stronger', 'JJR'),
 ('than', 'IN'),
 ('angled', 'VBN'),
 ('nailing', 'NN'),
 ('or', 'CC'),
 ('screw', 'JJ'),
 ('fastening', 'VBG'),
 ('alone', 'RB'),
 ('Help', 'NNP'),
 ('ensure', 'VB'),
 ('joints', 'NNS'),
 ('are', 'VBP'),
 ('consistently', 'RB'),
 ('straight', 'JJ'),
 ('and', 'CC'),
 ('strong', 'JJ'),
 ('Dimensions', 'NNS'),
 (':', ':'),
 ('3', 'CD'),
 ('in', 'IN'),
 ('.', '.'),
 ('x', 'JJ'),
 ('3', 'CD'),
 ('in', 'IN'),
 ('.', '.'),
 ('x', 'JJ'),
 ('1-1/2', 'JJ'),
 ('in', 'IN'),
 ('.', '.'),
 ('Made', 'VBN'),
 ('from', 'IN'),
 ('12-Gauge', 'JJ'),
 ('steel', 'NN'),
 ('Galvanized', 'NNP'),
 ('for', 'IN'),
 ('extra', 'JJ'),
 ('corrosion', 'NN'),
 ('resistance', 'NN'),
 ('Install', 'NNP'),

In [57]:
attr.head()

,name,value
product_uid,,
100001,Gauge,12
100001,MFG Brand Name,Simpson Strong-Tie
100001,Material,Galvanized Steel
100001,Multi,Versatile connector for various 90Â° connectio...
100001,Number of Pieces,1
